In [1]:
import torch
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import LabelEncoder
from restaurant_model import RestaurantReviewsModel
import joblib
# Load the saved model
model_path = 'restaurant_reviews_model.pth'  # Replace with the path to your saved model
num_categories= 5
num_polarities= 3

# Define the BERT model and tokenizer
bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert = BertModel.from_pretrained(bert_model_name)

model = RestaurantReviewsModel(bert, num_categories, num_polarities)  # Create an instance of your model
model.load_state_dict(torch.load(model_path))
model.eval()

# Load the saved label encoders
label_encoder_save_path = "label_encoders.pkl"  # Replace with the path to your saved label encoders
le_category = joblib.load(label_encoder_save_path + "_category")
le_polarity = joblib.load(label_encoder_save_path + "_polarity")

# Define your tokenizer and maximum text length
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_text_length = 128  # Define your desired maximum text length

# Define a function to preprocess and tokenize the text
def preprocess_text(text, tokenizer, max_length):
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return inputs

# Define the unseen sentences
unseen_sentences = [
    "The food was really good.",
    "The service was great.",
    "The ambiance of the restaurant is pleasant.",
    "I absolutely love this restaurant; everything about it is perfect.",
    "My experience at this restaurant was neither exceptional nor terrible.",
    "The restaurant occasionally has special discounts, which is a plus."
]

# Preprocess the unseen sentences and tokenize them
unseen_inputs = [preprocess_text(text, tokenizer, max_text_length) for text in unseen_sentences]

# Prepare input tensors for the unseen data
unseen_input_ids = torch.cat([inputs['input_ids'] for inputs in unseen_inputs], dim=0)
unseen_attention_mask = torch.cat([inputs['attention_mask'] for inputs in unseen_inputs], dim=0)
unseen_token_type_ids = torch.cat([inputs['token_type_ids'] for inputs in unseen_inputs], dim=0)

# Use the loaded model to make predictions on the unseen data
with torch.no_grad():
    category_logits, polarity_logits = model(unseen_input_ids, unseen_attention_mask, unseen_token_type_ids)

# Convert the model's predictions to actual labels
category_preds = le_category.inverse_transform(category_logits.argmax(1).cpu().numpy())
polarity_preds = le_polarity.inverse_transform(polarity_logits.argmax(1).cpu().numpy())

# Display the predictions
for i, sentence in enumerate(unseen_sentences):
    print(f"Sentence: {sentence}")
    print(f"Predicted Category: {category_preds[i]}")
    print(f"Predicted Polarity: {polarity_preds[i]}")
    print()


C:\Users\Charu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentence: The food was really good.
Predicted Category: food
Predicted Polarity: positive

Sentence: The service was great.
Predicted Category: service
Predicted Polarity: positive

Sentence: The ambiance of the restaurant is pleasant.
Predicted Category: ambience
Predicted Polarity: positive

Sentence: I absolutely love this restaurant; everything about it is perfect.
Predicted Category: miscellaneous
Predicted Polarity: positive

Sentence: My experience at this restaurant was neither exceptional nor terrible.
Predicted Category: miscellaneous
Predicted Polarity: negative

Sentence: The restaurant occasionally has special discounts, which is a plus.
Predicted Category: price
Predicted Polarity: positive

